<a href="https://colab.research.google.com/github/ReMeiradL/Apache_Beam/blob/main/C%C3%B3pia_de_apache_beam_ParDo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#INSTALADOR PARA MÁQUINA VIRTUAL, EM MÁQUINA LOCAL NÃO COLOQUE [INTERATIVE]
!pip install apache_beam[interative]

In [2]:
#IMPORT BIBLIOTECA
import apache_beam as beam

In [3]:
#beam.DoFn é usado para definir funções de transformação.
#e também herda funcionalidades dessa classe para métodos especificos.
class filtro(beam.DoFn):
  #process é obrigatório quando usamos DoFn, ele ativa o processamento.
  #self é uma convenção em Python para se referir à instância da própria classe.
  #record é uma variável que representa um elemento da PCollection
  #que está sendo processado pelo método process.
  def process(self, record):
    if int(record[8])>0:
      #retorna em uma lista
      return [record]

In [5]:
p1 = beam.Pipeline()

Tempo_Atrasos = (
    p1
    |"Importar dados Tempo" >> beam.io.ReadFromText('/content/voos_sample.csv',
                                              skip_header_lines = 1)
    |"Separar por vírgula Tempo" >> beam.Map(lambda record: record.split(','))
    #permite aplicar uma função personalizada (uma classe que herda de beam.DoFn)
    #a cada elemento da PCollection.
    |"Pegar voos atrasados Tempo" >> beam.ParDo(filtro())
    #Essa  de cima oimiza o código, substituindo essa linha:
    #|"Pegar voos atrasados Tempo" >> beam.Filter(lambda record: int(record[8]) > 0)
    #Criando uma Key e um Value, utilizando a coluna 4 (destino) e o atraso.
    |"Criar conjunto par Tempo" >> beam.Map(lambda record: (record[4], int(record[8])))
    # Soma todas as keys que são iguais, para um value unico
    |"Somar por key Tempo" >> beam.CombinePerKey(sum)
)

In [6]:
Qtd_Atrasos = (
    p1
    |"Importar dados QTD" >> beam.io.ReadFromText('/content/voos_sample.csv',
                                              skip_header_lines = 1)
    |"Separar por vírgula QTD" >> beam.Map(lambda record: record.split(','))
    #permite aplicar uma função personalizada (uma classe que herda de beam.DoFn)
    #a cada elemento da PCollection.
    |"Pegar voos atrasados QTD" >> beam.ParDo(filtro())
    #Criando uma Key e um Value, utilizando a coluna 4 (destino) e o atraso.
    |"Criar conjunto par QTD" >> beam.Map(lambda record: (record[4], int(record[8])))
    # Soma todas as keys que são iguais, para um value unico
    #Fornece a quantidade de atrasos por aeroporto
    |"Somar por key QTD" >> beam.combiners.Count.PerKey(sum)
)

In [7]:
tabela_atrasos = (
    #Criando tabela sendo dicionário.
    #Essa estrutura pode ser útil em determinados casos,
    #especialmente se você estiver agrupando ou processando dados
    #que têm uma relação específica entre eles.
    {'Qtd_Atrasos':Qtd_Atrasos, 'Tempo_Atrasos':Tempo_Atrasos}
    #É usada para agrupar esses dados por chave.
    #útil quando precisamos realizar operações de agregação ou processamento
    #em grupos específicos de dados.
    |"Group by" >> beam.CoGroupByKey()
    |"Mostrar dados" >> beam.Map(print)
)

p1.run()

('LAX', {'Qtd_Atrasos': [4], 'Tempo_Atrasos': [92]})
('HNL', {'Qtd_Atrasos': [1], 'Tempo_Atrasos': [15]})
('DFW', {'Qtd_Atrasos': [1], 'Tempo_Atrasos': [95]})
('OGG', {'Qtd_Atrasos': [1], 'Tempo_Atrasos': [138]})
('JFK', {'Qtd_Atrasos': [4], 'Tempo_Atrasos': [220]})
